In [1]:
import vtk
import numpy as np

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy import ndimage
import PIL
import json
from urbanComponent import *
import math

In [3]:
f = open('../../../public/data/manhattan_wo_cells/buildings_s_joined.json', 'r')

data = json.load(f)

joinedLayerIndex = 0

values = []

for joinedObject in data['joinedObjects']:
    if(joinedObject['joinedLayerIndex'] == joinedLayerIndex):
        values = joinedObject['otherValues']
        break

f.close()

uc = UrbanComponent()

uc.setWorkDir("../../../public/data/manhattan_wo_cells")
uc.addLayerFromJsonFile("../../../public/data/manhattan_wo_cells/buildings_s.json")

buildingsJson = uc.layers['json'][0]

# f = open('./buildings.json', 'w')

# f.write(json.dumps(buildingsJson))

# f.close()

latitude = []
longitude = []
height = []

for geometry in buildingsJson['data']:
    for i in range(int(len(geometry['geometry']['coordinates'])/3)):
        latitude.append(geometry['geometry']['coordinates'][i*3])
        longitude.append(geometry['geometry']['coordinates'][i*3+1])
        height.append(geometry['geometry']['coordinates'][i*3+2])

df = pd.DataFrame(data={'latitude': latitude, 'longitude': longitude, 'height': height})

m1 = df[['latitude','longitude','height']].to_numpy()[:,1]
m2 = df[['latitude','longitude','height']].to_numpy()[:,0]
m3 = df[['latitude','longitude','height']].to_numpy()[:,2]

xmin = m1.min()
xmax = m1.max()
ymin = m2.min()
ymax = m2.max()
zmin = m3.min()
zmax = m3.max()

values = np.array(values).reshape((len(values),))

wmin = values.min()
wmax = values.max()

# X, Y, Z, W = np.mgrid[xmin:xmax:100, ymin:ymax:100, zmin:zmax:50, wmin:wmax:0.1]

# X, Y, Z = np.mgrid[xmin:xmax:100, ymin:ymax:100, zmin:zmax:50]
X, Y, Z = np.mgrid[xmin:xmax:25, ymin:ymax:25, zmin:zmax:50]

# positions = np.vstack([X.ravel(), Y.ravel(), Z.ravel(), W.ravel()]) # grid positions

positions = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]) # grid positions

# data_points = np.vstack([m1, m2, m3, values]) 

data_points = np.vstack([m1, m2, m3]) 

kernel = scipy.stats.gaussian_kde(data_points, weights=values)

sampled_values = np.reshape(kernel(positions).T, X.shape)
# sampled_values = kernel(positions).T


(1779003,)


In [ ]:
#Create an instance of vtkImageData object
vtk_image = vtk.vtkImageData()

# Set the dimensions of the VTK image data object to match the shape of a NumPy array called `data`
vtk_image.SetDimensions(sampled_values.shape[2], sampled_values.shape[1], sampled_values.shape[0])

In [ ]:
# Set the origin of the VTK image data object to (0, 0, 0)
vtk_image.SetOrigin(0, 0, 0)

# Set the pixel spacing of the VTK image data object to (1, 1, 1)
vtk_image.SetSpacing(1, 1, 1)

# Allocate memory for the scalar data in the VTK image data object
vtk_image.AllocateScalars(vtk.VTK_FLOAT, 1)

# Retrieve a pointer to the scalar data in the VTK image data object
scalars = vtk_image.GetPointData().GetScalars()

# Copy the pixel data from a NumPy array called `data` into the VTK image data object
scalars.SetVoidArray(sampled_values, sampled_values.size, 1)

# Create an instance of a VTK XML image data writer
writer = vtk.vtkXMLImageDataWriter()

# Set the output filename for the VTK XML image data writer to a vti file
writer.SetFileName("grid.vti")

# Set the input data for the VTK XML image data writer to the VTK image data object that we created earlier
writer.SetInputData(vtk_image)

# Write the VTK XML file to disk using the input data and filename that we specified
writer.Write()